# Genome-wide association between YAP/TAZ/TEAD and AP-1 at enhancers drives oncogenic growth

https://pubmed.ncbi.nlm.nih.gov/26258633/

YAP and TAZ are proteins that act like __“go” signals__ for cells. When they’re active, cells grow, divide, and if things go wrong turn cancerous. They don’t bind DNA by themselves, they need helpers.

The Hippo pathway is basically the __cell’s growth brake system__. When Hippo is on, YAP/TAZ are kept quiet. When Hippo fails, YAP/TAZ run loose.

TEAD proteins are the __DNA-binding handles__. YAP/TAZ grab TEAD, and TEAD grabs DNA. No TEAD, no gene activation.

AP-1 (made of JUN and FOS) is another transcription factor. Think of AP-1 as a __stress-response and growth accelerator__ that turns on genes when cells are stimulated or damaged.

The authors asked: _How do YAP and TAZ really turn genes on? Not in theory. In real cancer cells, on real DNA._

They did ChIP-seq, which is basically __“where on the genome are these proteins sitting?”__

What they found is the key insight:

YAP/TAZ do not work alone with TEAD. Almost everywhere YAP/TAZ bind, __AP-1 is sitting right next to them on the DNA__.

Even more important:
The DNA regions contain two matching “__words__”:
• one that TEAD recognizes
• one that AP-1 recognizes

These are called __composite regulatory elements__. 
Translation: the DNA is written to require _both_ systems at once.

So what’s really happening?

YAP/TAZ + TEAD + AP-1 physically form a __team__ on the DNA. Together, they flip on genes much harder than any of them could alone.

What genes do they activate?

They activate genes that:
• push cells into __S-phase__ (DNA replication)
• push cells through __mitosis__ (cell division)

In other words: __the core cell-cycle engine__.

Where does this gene control happen?

Here’s a subtle but important point.

It mostly __does NOT happen at promoters__ (right at the gene start).
It happens at __distal enhancers__, which can be tens or hundreds of thousands of DNA letters away.

Those enhancers physically __loop through 3D space__ to touch the gene’s promoter. So YAP/TAZ aren’t whispering to genes directly, they’re pulling levers from across the room.

__Now the cancer relevance.__

When AP-1 activity is increased:
→ YAP/TAZ-driven tumor growth explodes.

When AP-1 is removed:
→ YAP/TAZ loses much of its cancer-driving power.

And the reverse is also true:

AP-1 by itself can drive skin cancer in mice.
But if YAP/TAZ are genetically deleted:
→ AP-1 can’t cause tumors anymore.

That means neither system is sufficient alone. They depend on each other.

The big takeaway:

Cancer growth here isn’t driven by a single “oncogene switch”.
It’s driven by __signal integration at the level of chromatin__.

YAP/TAZ bring mechanical and growth signals.
AP-1 brings stress and mitogenic signals.
The DNA itself demands that both show up together.

If you want the dumbest possible one-liner:

YAP/TAZ are the accelerator, AP-1 is the ignition, TEAD is the key, and cancer only starts when all three are in place at the same DNA locations.

Why this paper matters:

If you think “just inhibit YAP” or “just inhibit AP-1” will solve cancer, that’s naive. This paper shows the problem lives in __combinatorial control__, not single targets.

## Venn Diagram

It visualizes a high degree of co-occupancy between YAP and TAZ binding events.

It shows:

- Total number of YAP ChIP-seq peaks
- Total number of TAZ ChIP-seq peaks
- A __proxy count__ for “YAP peaks that overlap at least one TAZ peak”

That proxy is `n_overlap = length(YAP_overlap_TAZ_peaks)`.

So the biological claim encoded in the figure is:

_“Most YAP peaks overlap TAZ peaks, and most TAZ peaks overlap YAP peaks.”_

Bottom line:

Your Venn diagram shows that YAP and TAZ ChIP-seq peaks massively co-occur in the genome.

In [ ]:
# Big picture: what this analysis actually is:
# You are reproducing Figure 1 a, b, c from a ChIP-seq paper.
# Biological question:
  
#  - Do YAP and TAZ bind the same genomic regions?
  
#  - Do those shared regions also recruit TEAD4?
  
#  - Where is TEAD4 positioned relative to YAP/TAZ peak summits?

# Use GenomicRanges in R
# https://pubmed.ncbi.nlm.nih.gov/26258633/
# https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE66081

# if (!requireNamespace("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")

# BiocManager::install("ggplot2")

# Let’s use R instead with the `GenomicRanges` package. 
# Read my [blog post](https://divingintogeneticsandgenomics.com/post/genomic-interval/) on why we need to learn how to use it.

library(GenomicRanges)
library(rtracklayer) # for reading in bed file, bioConduct package, use to read in the peak files
library(here) # to give full path of the file

#=======================================================================

# ----- 1. Loading peak files → GenomicRanges ----- #

# What happens here:
#  - rtracklayer::import() reads a BED-like file
#  - It creates a GRanges object
#  - Each row = one peak
#  - Each peak has:
#    - seqnames (chromosome)
#    - ranges (start, end)
#    - metadata columns (score, signalValue, etc.)

#-----------------------------------------------------------------------

# TAZ_peaks <- import(here("1_reproduce_figures_using_Supplementary_Data/data/fastq/TAZ_peak/TAZ_peaks.narrowPeak"))
TAZ_peaks <- import("N:/Bioinformatik/Programming/Projects/1_reproduce_figures_using_Supplementary_Data/data/fastq/TAZ_peak/TAZ_peaks.narrowPeak")

# file.exists(here("data/fastq/TAZ_peak/TAZ_peaks.narrowPeak"))
# list.files(here("data/fastq/TAZ_peak"))

# list.files(
#   "N:/Bioinformatik/Programming/Projects",
#   pattern = "narrowPeak",
#   recursive = TRUE,
#   full.names = TRUE
# )

YAP_peaks <- import("N:/Bioinformatik/Programming/Projects/1_reproduce_figures_using_Supplementary_Data/data/fastq/YAP_peak/YAP_peaks.narrowPeak")
#YAP_peaks<- import(here("1_reproduce_figures_using_Supplementary_Data/data/fastq/YAP_peak/YAP_peaks.narrowPeak"))

TAZ_peaks # gives you the GenomicRanges object, 10719
YAP_peaks # gives you the GenomicRanges object,  9807

#=======================================================================

# ----- 2. Overlap logic: YAP vs TAZ ----- #

# Important conceptual point:
#  - subsetByOverlaps(A, B)
#    → “keep A peaks that overlap any peak in B”

# That means:
#  - length(A ∩ B) is not symmetric
#  - One peak can overlap multiple peaks in the other set

# That’s why you get:
#  - 7154 TAZ peaks overlapping YAP
#  - 7164 YAP peaks overlapping TAZ


# Hidden assumption:
#  - “Overlap” means ≥1 bp
#  - No distance threshold
#  - No summit logic yet

# ----------------------------------------------------------------------

# We have `r length(YAP_peaks)` YAP peaks and `r length(TAZ_peaks)` TAZ peaks.
# How many of them overlap?
TAZ_overlap_YAP_peaks <- subsetByOverlaps(TAZ_peaks, YAP_peaks)
length(TAZ_overlap_YAP_peaks) # 7154

TAZ_overlap_YAP_peaks # look at it

YAP_overlap_TAZ_peaks <- subsetByOverlaps(YAP_peaks, TAZ_peaks)
length(YAP_overlap_TAZ_peaks) # 7164

YAP_overlap_TAZ_peaks # look at it

# They are not exactly the same because some peaks overlap with multiple peaks
# from the other set.

# so we have 7154 out of 10719 TAZ peaks overlapping with YAP1 peaks.
# (Note, this is the same number we got from bedtools intersect).

# and we have 7164 out of 9807 YAP1 peaks overlapping with TAZ peaks.

# The venn-diagram needs a common number in the intersection.
# How do we deal with it? There are different decisions you can make
# and it does not affect the conclusion of the figure:
#  most of the TAZ and YAP1 peaks overlap!

# We can just use the number of YAP1 peaks that overlap with TAZ as the intersection.

#=======================================================================

# ----- 3. The Venn diagram problem -----

# There are many packages to make a venndiagram. I use [Vennerable.](https://github.com/js229/Vennerable)

# devtools::install_github("js229/Vennerable")

#install.packages("devtools")
#  library(devtools)

# install.packages("BiocManager")
#BiocManager::install(version = "3.19")
#  BiocManager::install(version = "3.22")

# BiocManager::install(c("graph", "RBGL"))

# Sys.which("make")

# devtools::install_github("js229/Vennerable")
# remotes::install_github("js229/Vennerable")
library(Vennerable)


# Venn diagrams want one number for the intersection.

# Reality:
#  - Overlaps are many-to-many
#  - Biology doesn’t care about your plotting library


n_YAP <- length(YAP_peaks)  # Total number of peaks for each Set
n_TAZ <- length(TAZ_peaks)  # Total number of peaks for each Set


# This is a reasonable simplification, and the paper explicitly allows it.
# Key idea:
#   The conclusion (“most peaks overlap”) is robust to the exact counting choice.
n_overlap <- length(YAP_overlap_TAZ_peaks)

venn_data <- Venn(SetNames = c("αYAP peaks", "αTAZ peaks"),
                  Weight = c(
                    # 1 = only in set 1, but not in set 2
                    "10" = n_YAP, # Unique to A
                    # 2 = only in set 2, but not in set 1
                    "01" = n_TAZ, # Unique to B
                    # common to both sets
                    "11" = n_overlap         # Intersection
                  ))


# Plot the Venn diagram
plot(venn_data)

# You can also use makeVennDiagram in the ChIPpeakAnno package.

# In our case, we already have the number of the two sets and the intersection, 
# so we used vennerable.

# Take a look at [ggVennDiagram](https://github.com/gaospecial/ggVennDiagram) too


In [ ]:
##### Figure 1 b
# We can easily make Figure 1b now that we have some foundations.

# TEAD4_peak <- import(here("1_reproduce_figures_using_Supplementary_Data/data/fastq/TEAD4_peak/TEAD4_peaks.narrowPeak"))
TEAD4_peak <- import(here("N:/Bioinformatik/Programming/Projects/1_reproduce_figures_using_Supplementary_Data/data/fastq/TEAD4_peak/TEAD4_peaks.narrowPeak"))

#=======================================================================

# ----- 4. Adding TEAD4: second-order overlap -----

# Now the logic stack is:
#  1. Start with YAP peaks
#  2. Keep only those overlapping TAZ
#  3. From those, keep only those overlapping TEAD4

# This answers:
# “Among YAP/TAZ shared sites, how many recruit TEAD4?”

YAP_overlap_TAZ_peaks_overlap_TEAD4<- subsetByOverlaps(YAP_overlap_TAZ_peaks, TEAD4_peak)

n_YAP_TAZ <- length(YAP_overlap_TAZ_peaks)  # Total peaks 
n_TEAD4 <- length(TEAD4_peak)  # Total peaks 
n_overlap2 <- length(YAP_overlap_TAZ_peaks_overlap_TEAD4)

venn_data2 <- Venn(SetNames = c("YAP/TAZ", "TEAD4"),
                  Weight = c(
                    "10" = n_YAP_TAZ, # Unique to A
                    "01" = n_TEAD4, # Unique to B
                    "11" = n_overlap2        # Intersection
                  ))

# Plot the Venn diagram
plot(venn_data2)

In [ ]:
# ----- 5. Why summits matter (Figure 1c) -----

# Up to now:
# - You treated peaks as intervals

# Now:
#  - You treat peaks as points (summits)

# Why?
#  - Peak width varies wildly
#  - Binding precision lives at the summit
#  - Distance between summits is biologically meaningful

##### Figure 1c

# This figure requires a little more work. Let’s decompose it.

# Description of the figure in the paper:

# Position of TEAD4 peak summits relative to the summits of the overlapping YAP/TAZ peaks, 
# in a 500 bp window surrounding the summit of YAP/TAZ peaks.

# TAZ peaks coordinates and summit positions were used to represent common peaks 
# between YAP and TAZ peaks (YAP/TAZ peaks) and were used when comparing 
# YAP/TAZ peaks with other ChIP-seq data.

# - data we need: the TAZ peak set and the TEAD4 peak set.
# - x-axis: when a TEAD peak overlaps with a TAZ peak, 
#   the distance between the summit of the the TAZ peak and the TEAD4 peak summit.
# - y-axis: the number of TEAD peaks for each distance

# -------------------------------------------------------
# Each summit:
#  - Is a 1 bp interval
#  - Represents max ChIP signal

# A summit is the highest signal point within a peak. MACS3 outputs that.
#TAZ_summit<- import(here("data/fastq/TAZ_peak/TAZ_summits.bed"))
TAZ_summit<- import("N:/Bioinformatik/Programming/Projects/1_reproduce_figures_using_Supplementary_Data/data/fastq/TAZ_peak/TAZ_summits.bed")

# ----- 6. Restricting to biologically relevant summits -----
# Only keep TAZ summits from peaks that overlap YAP.
# So:
#  - You define YAP/TAZ peaks
#  - Using TAZ coordinates as the reference frame

# we will only subset the summit that is overlapping with YAP
## To make that line plot:
# 1. make sure two peaks first overlap with each other
# 2. calculate the summit distances between those two overlapping peaks
# TAZ_summit$name is <character> name
# name is the same name to subset the TAZ_summit
TAZ_summit<- TAZ_summit[
  TAZ_summit$name %in% TAZ_overlap_YAP_peaks$name
  ]

# TEAD4_summit<- import(here("data/fastq/TEAD4_peak/TEAD4_summits.bed"))
TEAD4_summit<- import("N:/Bioinformatik/Programming/Projects/1_reproduce_figures_using_Supplementary_Data/data/fastq/TEAD4_peak/TEAD4_summits.bed")

TEAD4_summit

# They both represent a single base point that has the highest signal in the peaks.
# -------------------------------------------------------

# ----- 7. Creating a ±250 bp window around TAZ summits -----
# This creates:
#  - A symmetric window
#  - From −250 bp to +250 bp
#  - Centered on TAZ summit

# expand the TAZ summit to a 500bp window
# because x-axis -250 to 250
# 250bp upstream and 250bp upstream
TAZ_500bp_window<- resize(TAZ_summit, width = 500, fix="center")

# ----- 8. Matching TEAD4 summits to TAZ windows -----
# Which TEAD4 summits fall into which TAZ windows
# One TEAD4 summit can match multiple TAZ windows
# One TAZ window can contain multiple TEAD4 summits
# findOverlaps of TEAD4_summit with TAZ_500bp_window
hits<- findOverlaps(TEAD4_summit, TAZ_500bp_window)

# a hits object with the indices of the overlapping query and subject
hits

# queryHits(hits) & subjectHits(hits) to extract the numbers
# calculating the distance between those two summits
# head(summit_distance)
summit_distance<- distance(TEAD4_summit[queryHits(hits)], TAZ_summit[subjectHits(hits)])

table(summit_distance)

# look at the distance which is equal to 0 & look at the TEAD4 summit
TEAD4_summit[queryHits(hits)][summit_distance ==0]

TAZ_summit[subjectHits(hits)][summit_distance ==0]

# The built-in `distance` function returns the pair-wise distance in absolute value.

# Let’s revise it to return negative values when TEAD4 summit precede the 
# TAZ summit and positive values when TEAD4 summit follows TAZ summit.


# ----- 9. Distance calculation (unsigned → signed) -----

## Unsigned distance
# summit_distance<- distance(TEAD4_summit[queryHits(hits)], TAZ_summit[subjectHits(hits)])

## Signed distance

# Interpretation:
#  - Negative → TEAD4 upstream of TAZ
#  - Positive → TEAD4 downstream of TAZ
#  - Zero → exact summit coincidence

# Compute signed distances
# create a new function
signed_distance <- function(A, B) {
  # Compute unsigned distance
  dist <- distance(A, B)
  
  # but if A is smaller than B
  # A is smaller than B and B-
  # If A is greater than B
  # Determine signs based on whether A precedes or follows B
  sign <- ifelse(start(A) < start(B), -1, 1)
  
  # Apply sign to distance
  dist * sign
}

library(dplyr)
library(ggplot2)
# summit_distance is a numeric vector
# Each element is a distance in base pairs
# Each number = one TEAD4–TAZ summit pair.
summit_distance<- signed_distance(TEAD4_summit[queryHits(hits)],
                                  TAZ_summit[subjectHits(hits)])
# table(summit_distance)
# Interpretation:
#  - Distance −249 bp occurred 1 time
#  - Distance -246 bp occurred 2 times
#  - Distance -233 bp occurred 1 time
#  - Distance -232 bp occurred 2 times

# So now you have:
#  - x-axis: distance
#  - y-axis: count (density proxy)

# table() returns a special object, not a normal data frame.

# class(table(summit_distance))

# A table:
#  - Looks like a matrix
#  - Behaves like a named vector
#  - Is awkward for ggplot2

# ggplot2 wants a data frame:
#  - one column per variable
#  - one row per observation

# So we need to convert!!!

# ----- 10. Turning distances into a density plot -----
# This converts:
#  - “many individual distances”
#     → “count per distance value”

# Then:
#  - Sort x-axis
# - Plot counts vs distance

# “Count how many summit pairs occur at each distance,
# then store the result in a tidy table so I can plot it.”

# This produces the raw spike plot, which is noisy.
# table() answers one question:
# “How many times does each value occur?”

# What %>% (the pipe) means
# “Take the result from the left and pass it as the first argument to the function on the right.”
distance_df<- table(summit_distance) %>%
  tibble::as_tibble() 
# So this is equivalent to:
  # tibble::as_tibble(table(summit_distance))

# What as_tibble() does
# tibble is a modern version of data.frame.
# tibble::as_tibble(table(summit_distance))
# Converts to:
# A tibble:
#   summit_distance     n
#   <chr>           <int>
# 1 -12                 2
# 2 0                   3
# 3 5                   3
# 4 10                  1

# Two columns:
#  - summit_distance → the distance (as character for now)
#  - n → how many times it occurred

# This format is exactly what ggplot() wants.

# Why summit_distance becomes character
# <summit_distance> <chr>
# Because:
#  - table() stores its names as strings
#  - as_tibble() preserves that

# That’s why later you had to do:
# mutate(summit_distance = as.numeric(summit_distance))

distance_df

#class(distance_df)
#str(distance_df)

# X-Axis | Y-Axis

# Plot it:
# It is exactly equivalent to:
# ggplot(distance_df, ...)

# It creates a plot object that says:
# “I intend to plot something using this data.
# The x-variable will be summit_distance.
# The y-variable will be n.”

distance_df %>%
  # aes() = aesthetic mapping
  # Map the column summit_distance to the x-axis
  # Map the column n to the y-axis
  ggplot(aes(x=summit_distance, y = n)) +
  geom_point()

# Hmm, something is off… the summit distance on the x-axis needs to be reordered

distance_df %>%
  # summit_distance (character because of table())
  # mutate() = add or transform columns in a data frame (or tibble).
  # Here, you’re overwriting summit_distance to be numeric.
  # class(distance_df$summit_distance)  # before: "character"
  mutate(summit_distance = as.numeric(summit_distance)) %>%
  # Sorts the tibble by summit_distance ascending.
  # Result: rows go from the most negative distance to the most positive distance.
  arrange(summit_distance) %>%
  # class(distance_df$summit_distance)  # after mutate: "numeric"
  # Why important:
  #  - ggplot now treats x-axis as continuous instead of categorical.
  #  - The points will be positioned correctly along the genome, not evenly spaced like labels.
  
  # n (integer count of how many TEAD4–TAZ summit pairs are at each distance)
  # ggplot() creates a plot object.
  # aes() = aesthetic mapping (what goes where)
  # x = summit_distance → genomic distance relative to TAZ peak summit
  # y = n → number of TEAD4 summits at that distance
  ggplot(aes(x=summit_distance, y = n)) +
  # Actually draws a point for every row in the tibble:
  # x-coordinate = summit_distance (numeric now)
  # y-coordinate = n
  # Effectively: each dot = “TEAD4 summits count at this distance from TAZ peak.”
  geom_point()

# Let’s connect the points with a line:
# Take distance_df
distance_df %>%
  # Convert the distance column to numeric (so it’s a real axis)
  mutate(summit_distance = as.numeric(summit_distance)) %>%
  # Sort the table so distances go from negative to positive
  arrange(summit_distance) %>%
  # Start a plot: x = distance, y = count
  ggplot(aes(x=summit_distance, y = n)) +
  # insdtead of geom_point() we use geom_line()
  # Draw one point per distance-count pair
  # This gives you a scatter plot of TEAD4 density around TAZ summits.
  geom_line()

# Result: super noisy / “wiggly” because counts jump a lot between neighboring base pairs

# ----- 11. Binning to smooth signal (5 bp bins) -----

# Groups distances into 5 bp bins
# Averages counts per bin
# Reduces noise
# Reveals structure

# This is effectively a discrete kernel smoothing without saying the word.

# Biologically:
#  - We don’t care about single-bp jitter
#  - We care about positional bias at ~10 bp scale

# The final plot answers:
# “Is TEAD4 centered on YAP/TAZ summits?”

# And the answer is: yes, very strongly.

# The plot looks too wigglely. Let’s smooth it by average the number of peaks per 5 bp bin.
df_binned <- distance_df %>%
  # Step 1: Convert distances to numeric & arrange
  mutate(summit_distance = as.numeric(summit_distance)) %>%
  arrange(summit_distance) %>%
  # summit_distance / 5
  # Divide each distance by 5 → groups of 5 bp (e.g., −12/5 = −2.4)
  # floor()
  # Round down to nearest integer (−2.4 → −3)
  # Ensures all distances in the same 5 bp interval map to the same bin
  # * 5
  # Convert back to original scale
  # −3 → −15, meaning bin covers −15 to −11
  # Result: a new column called bin, grouping distances into 5 bp windows:
  #   summit_distance	|   bin
  #         -12	      |   -15
  #         -11       | 	-15
  #         -10	      |   -10
  #         -9	      |   -10
  mutate(bin = floor(summit_distance / 5) * 5) %>%  # Create bins by grouping every 5 bp
  # Group by bin & summarize
  # group_by(bin) → treat all rows in the same 5 bp bin as one group
  group_by(bin) %>%
  # summarise(n = mean(n, na.rm=TRUE)) → compute average count in each bin
  # na.rm = TRUE ignores missing values
  # Result: one row per 5 bp bin, with smoothed y-values
  # summarise(n = mean(n, na.rm = TRUE))  # Calculate average 'n' for each bin
  summarise(n = mean(n, na.rm = TRUE)) %>%
  # Example:
  #    bin	|    n (mean)
  #   -250	|    0.6
  #   -245	|    1.2
  #   -240	|    1.0
  #    ...	|    ...
  # Use [[ ]] to force the column
  mutate(n = (n / max(n)) *2)   # scale 0–1
  # Effect: the wiggles are smoothed, trend is clear, small fluctuations are averaged out.

# Groups summit_distance values into 5 bp bins by dividing by 5, taking the floor (rounding down), and multiplying back by 5 to get the bin lower bound.
# View the binned dataframe
print(df_binned)

# 4: Plot the smoothed data
df_binned %>%
  # x-axis = 5 bp bins
  # y-axis = average number of peaks in that bin
  ggplot(aes(x=bin, y = n)) +
  # Connect points with a line → shows trend of TEAD4 density around TAZ summits
  geom_line(color = "blue") +
  # Only label x-axis at −250, 0, +250 bp for readability
  scale_x_continuous(
    breaks = c(-250, 0, 250),
    expand = c(0.02, 0)
  ) +
  scale_y_continuous(
    limits = c(0, 2.125),
    breaks = seq(0, 2.125, 0.5),
    # sprintf("%.1f", x) → formats every number with 1 decimal place
    # So 0 → 0.0, 0.5 → 0.5
    labels = function(x) sprintf("%.1f", x) %>% 
      # sub("\\.0$", "", .) → removes the .0 at the end
      # So 0.0 becomes 0, 1.0 becomes 1
      sub("\\.0$", "", .),  # remove .0 from whole numbers
    expand = c(0, 0) # start exactly at 0, no padding
  ) +
  # Human-readable axis labels
  xlab("Distance to the summit \nof TAZ peaks (bp)") +
  ylab("Peak density (x10²)") +
  # Clean background, larger font
  theme_classic(base_size = 14) +
  theme(panel.border = element_rect(fill = NA))  # keeps top/right borders

# Scatter Plot

Data needed:

- we need the YAP, TAZ, and TEAD4 signal. This is the number of sequencing reads
- x-axis: TEAD4 signal
- y-axis: YAP1 signal

###### d. Linear correlation between the signal of YAP or TAZ and TEAD4 peaks in the 5522 shared binding sites. r2 is the coefficients of determination of the two correlations.

1. get the overlap of those three peak sets
2. count how many agrees in each of overlapping peaks
3. make scattered plot

## 1. Download fastq files

https://crazyhottommy.github.io/reproduce_genomics_paper_figures/01_download_fastq_from_GEO.html

We will use fastq-dl to download the fastq files from GEO.

https://github.com/rpetit3/fastq-dl/tree/master

`conda create -n fastq_download -c conda-forge -c bioconda fastq-dl`
`conda activate fastq_download`

Tip, conda is very slow. use mamba as a drop-in replacement for conda.

The relationship of Experiment to Run is a 1-to-many relationship, or there can be many Run accessions associated with a single Experiment Accession (e.g. re-sequencing the same sample). Although in most cases, it is a 1-to-1 relationship, you can use –group-by-experiment to merge multiple runs associated with an Experiment accession into a single FASTQ file.

and we see many SRR number for the same SRX sample.

`fastq-dl --accession SRX883576  --group-by-experiment`

Also, to reduce the size of the files. I will only take one SRR for the same SRX sample. or you will have to download all the SRR fastq for the same SRX sample and concatenate them together.

cd data/fastq<br>
// if you do not have wget <br>
// conda install wget <br>
`wget -nc ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR181/007/SRR1810907/SRR1810907.fastq.gz`<br>
`wget -nc ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR181/008/SRR1810918/SRR1810918.fastq.gz`<br>
`wget -nc ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR181/000/SRR1810900/SRR1810900.fastq.gz`<br>
`wget -nc ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR181/002/SRR1810912/SRR1810912.fastq.gz`<br>

### rename the files

`mv SRR1810907.fastq.gz TAZ.fastq.gz`<br>
`mv SRR1810900.fastq.gz YAP.fastq.gz`<br>
`mv SRR1810918.fastq.gz TEAD4.fastq.gz`<br>
`mv SRR1810912.fastq.gz IgG.fastq.gz`<br>

fastq files are just text files with 4 lines for a single record.

`zless -S YAP.fastq.gz`

### fastqc look at the quality of the reads

`conda create -n reproduce_figure fastqc -c bioconda`
`conda activate reproduce_figure`

In [ ]:
for fq in *fastq.gz
do
  fastqc $fq
done

Tip use multiQC

If there is low quality bases and adaptor contamination, use fastp to trim them off.



## 2. Align fastq to genome

### Where to download the reference?

watch my YouTube video on this topic:

https://invidious.nerdvpn.de/eIVlSG11umQ?autoplay=0&t=2

One can go to: 
1. UCSC https://hgdownload.soe.ucsc.edu/downloads.html

2. NCBI https://www.ncbi.nlm.nih.gov/datasets/docs/v2/policies-annotation/genomeftp/

3. ENSEMBL https://useast.ensembl.org/info/data/ftp/index.html

4. GENCODE https://www.gencodegenes.org/

to download the reference genome manually or use refgenie

### Choose the aligner

It is single-end read with 50 base pairs.

`zless TAZ.fastq.gz | head -2 |tail -1`
ATAGGCTTTAAGCTGTCTTTGGTTTGAAGGTGGGATTTTACCGGGGACCC <br>

`zless TAZ.fastq.gz | head -2 |tail -1 | wc -L`<br>
50

The ChIP-seq signal can happen to anywhere in the genome, so we need an aligner to align the reads to the genome.

For RNAseq data, we need to align the reads to a transcriptome.

STAR is a popular aligner for RNAseq.

BWA is very popular for DNA-seq (WGS, WES) alignment.

For long reads (pacbio or nanopore), use minimap2.

BWA and minimap2 are both developed by Heng Li, not surprisingly, the mapping God.

Let’s use bowtie2 https://bowtie-bio.sourceforge.net/bowtie2/index.shtml for our ChIP-seq data.

Bowtie 2 is an ultrafast and memory-efficient tool for aligning sequencing reads to long reference sequences. It is particularly good at aligning reads of about 50 up to 100s or 1,000s of characters, and particularly good at aligning to relatively long (e.g. mammalian) genomes.

In the early days, ChIP-seq data were 36bp long, so Bowtie 1 was used.

In [ ]:
# install bowtie2
conda install bowtie2 -c bioconda

### download index

The aligners usually needs an index file that is created using the genome reference files (fasta) we downloaded from UCSC, NCBI, ENSEMBL or GENCODE.

For bowtie2, there is a pre-built index file for us to download.

Scroll down the page https://bowtie-bio.sourceforge.net/bowtie2/index.shtml and on the left you will see the index to download.

We will download the GRCH38 (aka, hg38 without decoy). click it, it will download a GRCh38_noalt_as.zip file of 3.5G. unzip it and place the folder to the data/reference folder.

Which reference genome to use? Bonus, read this post by HengLi.

If there is no pre-built index, take a look at this nf-core/references.

In [ ]:
# make sure you are in the data folder
cd data/

# cd data/reference/GRCh38_noalt_as/files
# cd data/fastq/YAP.fastq.gz

bowtie2 -x reference/GRCh38_noalt_as/GRCh38_noalt_as -U fastq/YAP.fastq.gz -S fastq/YAP1.sam --threads 8  --no-mixed --no-discordant -k 1

`-x reference/GRCh38_noalt_as/GRCh38_noalt_as`: The path to the Bowtie2 reference genome index (built with bowtie2-build reference.fa reference_index).

`-U input.fastq`: Input FASTQ file for single-end reads.

`-S output.sam`: Output alignment file in SAM format.

`--threads 8`: Use 8 threads to speed up alignment.

`--no-mixed`: Ensures that only properly paired reads are reported (not relevant for single-end data but good for ChIP-seq).

`--no-discordant`: Prevents reporting discordant alignments (not relevant for single-end data but ensures consistent results for paired-end).

`-k 1`: Reports only the best alignment for each read (important for ChIP-seq to avoid multi-mapping).

Below is the output. And it is super-fast!! for 24 million reads, it only took me 1.5 mins with 8 CPUs.

In [ ]:
24549590 reads; of these:
  24549590 (100.00%) were unpaired; of these:
    895629 (3.65%) aligned 0 times
    23653961 (96.35%) aligned exactly 1 time
    0 (0.00%) aligned >1 times
96.35% overall alignment rate
(reproduce_figure)

The output is the sam file which is a format to store the alignment. It is just a text file and you can use `less -S YAP1.sam` to see the content.

After we made one sample work, we can loop over and do the same for other fastq files.

cd fastq<br>
less -S YAP1.sam

In [ ]:
@HD     VN:1.5  SO:unsorted     GO:query
@SQ     SN:chr1 LN:248956422
@SQ     SN:chr2 LN:242193529
@SQ     SN:chr3 LN:198295559
@SQ     SN:chr4 LN:190214555
@SQ     SN:chr5 LN:181538259
@SQ     SN:chr6 LN:170805979
@SQ     SN:chr7 LN:159345973
@SQ     SN:chr8 LN:145138636
@SQ     SN:chr9 LN:138394717
@SQ     SN:chr10        LN:133797422
@SQ     SN:chr11        LN:135086622
@SQ     SN:chr12        LN:133275309
@SQ     SN:chr13        LN:114364328
@SQ     SN:chr14        LN:107043718
@SQ     SN:chr15        LN:101991189
@SQ     SN:chr16        LN:90338345
@SQ     SN:chr17        LN:83257441
@SQ     SN:chr18        LN:80373285
@SQ     SN:chr19        LN:58617616
@SQ     SN:chr20        LN:64444167
@SQ     SN:chr21        LN:46709983

After we made one sample work, we can loop over and do the same for other fastq files.



In [ ]:
rm YAP.sam

for fq in fastq/*fastq.gz
do
  bowtie2 -x reference/GRCh38_noalt_as/GRCh38_noalt_as -U $fq -S ${fq/fastq.gz/sam} --threads 8  --no-mixed --no-discordant -k 1
done

here we used bash string manipulation ${fq/fastq.gz/sam} to replace the fastq.gz to sam as the output.

Bonus: read my blog post on it https://crazyhottommy.blogspot.com/2015/07/string-manipulation-in-bash.html

Make sure you have all the sam files:

In [ ]:
ls fastq/*sam
fastq/IgG.sam   fastq/TAZ.sam   fastq/TEAD4.sam fastq/YAP.sam

### convert sam to bam

Most of the tools work with the binary form of the sam file which is a bam file. The size of the bam file is smaller samtools is the key toolkit to deal with it.

In [ ]:
conda install samtools -c bioconda

for sam in fastq/*sam
do
  samtools view -bS $sam > ${sam/sam/bam}
done

In [ ]:
With percentage

# install pv

count=0
total=$(ls fastq/*.sam | wc -l)

for sam in fastq/*.sam
do
  count=$((count+1))
  echo "[$count/$total] Converting $sam → BAM"
  pv "$sam" | samtools view -b - > "${sam%.sam}.bam"
done



In [ ]:
# check the size of the sam and bam files
ls -shlt fastq/*am

In [ ]:
# remove the sam files to save space
rm fastq/*sam

sort the reads in the bam file by coordinates.

In [ ]:
for bam in fastq/*bam
do
  samtools sort -@ 4 -o ${bam/bam/sorted.bam} $bam
done

In [ ]:
with percentage

for bam in fastq/*.bam
do
  out="${bam%.bam}.sorted.bam"
  echo "Sorting $bam → $out"
  pv "$bam" | samtools sort -@ 4 -o "$out" -
done

In [ ]:
ls fastq/*bam

In [ ]:
# remove all the unsorted bam 
ls fastq/*bam | grep -v sorted.bam | xargs rm

Always test the command for a single sample and then apply it to all samples after you confirm it works.

In [ ]:
samtools sort -@ 4 \
  -o fastq/YAP.sorted.bam \
  fastq/YAP.bam


In [ ]:
samtools quickcheck fastq/YAP.sorted.bam
samtools idxstats fastq/YAP.sorted.bam | head


Why Sort a BAM File?

Efficient Retrieval by Genomic Coordinates:
Sorting arranges the reads in the BAM file by their genomic coordinates (e.g., by chromosome and position). This is crucial for downstream tools (e.g., variant calling, visualization) that rely on efficient access to reads aligned to specific regions. Compatibility with Analysis Tools:

Many tools require sorted BAM files as input because they process data regionally (e.g., calling peaks or calculating coverage).

### index the bam file

In [ ]:
for bam in fastq/*sorted.bam
do 
  samtools index $bam
done

In [ ]:
count=0
total=$(ls fastq/*sorted.bam | wc -l)

for bam in fastq/*sorted.bam
do
  count=$((count+1))
  echo "[$count/$total] Indexing $bam"
  samtools index "$bam"
done


Why Index a BAM File?

Random Access to Reads:
The BAM index (.bai or .csi file) allows tools to quickly locate and access reads aligned to specific genomic regions without scanning the entire file. This dramatically improves performance for tasks like visualization (e.g., in IGV) or focused analyses of particular loci.

Faster Processing:
Many tools (e.g., samtools, deepTools, GATK) use the index to load only relevant portions of the BAM file, reducing memory and computation time.

Requirement for Visualization:
Genome browsers like IGV or UCSC Genome Browser require indexed BAM files to display aligned reads efficiently.

Two things can be done here now:

- Generate a BigWig file
- Core Peaks

## Generate bigwig files and peak calling

### generate bigwig files

Now that we have the sorted bam file, we can generate a genome wide signal view as a bigwig file.

A BigWig file is a binary file format commonly used in bioinformatics to efficiently store and visualize continuous data over genomic coordinates, such as read coverage or signal intensity. It is derived from the Wiggle (WIG) format but optimized for faster access and reduced file size, making it suitable for large-scale genomic data.

UCSC has a detailed page explaining what is it https://genome.ucsc.edu/goldenpath/help/bigWig.html

Bonus: Bioinformatics is nortorious about different file formats. read https://genome.ucsc.edu/FAQ/FAQformat.html for various formats definitions

For this purpose, we will use deeptools https://deeptools.readthedocs.io/en/develop/content/example_usage.html

deeptools is very versatile and it has many [sub-commands](https://deeptools.readthedocs.io/en/develop/content/list_of_tools.html).



We will use [bamCoverage](https://deeptools.readthedocs.io/en/develop/content/tools/bamCoverage.html) to generate bigwig files.

In [ ]:
# install it via conda if you do not have it yet
conda install -c conda-forge -c bioconda deeptools
# or 
pip install deeptools

I really like the demonstration of how coverage files are computed by the deeptools [authors](https://docs.google.com/file/d/0B8DPnFM4SLr2UjdYNkQ0dElEMm8/edit?resourcekey=0-7YZ1j0PIefw22P18GFlUjg).

[Image](https://crazyhottommy.github.io/reproduce_genomics_paper_figures/imgs/bam2bigwig1.png)

In [ ]:
cd data/fastq
bamCoverage --bam YAP.sorted.bam --normalizeUsing RPKM --extendReads 200 -o YAP1.bw

Read [here](https://github.com/crazyhottommy/ChIP-seq-analysis/blob/master/part1.2_convert_bam2_bigwig.md) to understand why you need to extend the reads to 200 bp. We only sequenced 50bp of the DNA after antibody pull-down, but the real DNA is about 200 bp (the size of the DNA after sonication/fragmentation)

We used default bin size 50 bp.

#### Generate bigwig files for all samples

In [ ]:
for bam in *sorted.bam
do
  bamCoverage --bam $bam --normalizeUsing RPKM --extendReads 200 -o ${bam/sorted.bam/bw}
done

In [ ]:
bamCoverage --bam YAP.sorted.bam --normalizeUsing RPKM --extendReads 200 -o YAP1.bw
bamCoverage --bam IgG.sorted.bam --normalizeUsing RPKM --extendReads 200 -o IgG.bw
bamCoverage --bam TAZ.sorted.bam --normalizeUsing RPKM --extendReads 200 -o TAZ.bw
bamCoverage --bam TEAD4.sorted.bam --normalizeUsing RPKM --extendReads 200 -o TEAD4.bw

In [ ]:
count=0
total=$(ls *sorted.bam | wc -l)

for bam in *sorted.bam
do
  count=$((count+1))
  out="${bam%.sorted.bam}.bw"

  echo "[$count/$total] bamCoverage on $bam"

  bamCoverage \
    --bam "$bam" \
    --normalizeUsing RPKM \
    --extendReads 200 \
    -o "$out"
done


In [ ]:
count=0
total=$(ls *sorted.bam | wc -l)

for bam in *sorted.bam
do
  count=$((count+1))
  out="${bam%.sorted.bam}.bw"
  size=$(stat -c%s "$bam")

  echo "[$count/$total] bamCoverage on $bam"

  pv -s "$size" "$bam" > /dev/null &
  PV_PID=$!

  bamCoverage \
    --bam "$bam" \
    --normalizeUsing RPKM \
    --extendReads 200 \
    -o "$out"

  kill $PV_PID 2>/dev/null
done

In [ ]:
ls *bw

You can load .bw to IGV and look at the Signal.

1. Open IGV
2. Select Load from File
3. Select .bw files

Each bin is the bin size, 50 base pair bin size default.
Histogram.

BigWig file = RAW signals

### peak calling

Once you have the sorted.bam files, now you can call __peaks__.

__peaks__ = we have the RAW signals in BigWig file.
You want to identify all these regions in the genome computationally. <br>
That is what `MACS2` or `MACS3`is doing.

MACS is the most popular peak caller for ChIPseq. It is maintained by Tao Liu. I had the pleasure working with him on some single-cell ATACseq stuff when I was in Shirley Liu’s lab.

MACS is now MACS3! https://macs3-project.github.io/MACS/docs/INSTALL.html

In [ ]:
pip install --upgrade macs3

The MACS3 callpeak subcommands have many paramters and you want to read https://macs3-project.github.io/MACS/docs/callpeak.html

In [ ]:
macs3 callpeak -t YAP.sorted.bam -c IgG.sorted.bam -f BAM -n YAP -g hs --outdir YAP_peak

-t = treatment
-c = control (IgG control file)
-f = format
-n = name of folder
-g = genome is hs (human)
--outdir = YAP_peak


It takes a couple of minutes. output:

In [ ]:
ls YAP_peak/
YAP1_model.r          YAP1_peaks.narrowPeak YAP1_peaks.xls        YAP1_summits.bed

Do it for all samples.

In [ ]:
for bam in *sorted.bam
do
  if [[ "$bam" != "IgG.sorted.bam" ]]; then
    echo macs3 callpeak -t $bam -c IgG.sorted.bam -f BAM -n ${bam%.sorted.bam} -g hs --outdir ${bam/.sorted.bam/_peak}
  fi
done

In [ ]:
remove the “echo” and run it:

for bam in *sorted.bam
do
  if [[ "$bam" != "IgG.sorted.bam" ]]; then
    macs3 callpeak -t $bam -c IgG.sorted.bam -f BAM -n ${bam%.sorted.bam} -g hs --outdir ${bam/.sorted.bam/_peak}
  fi
done

In [ ]:
# modded

count=0
total=$(ls *sorted.bam | grep -v '^IgG.sorted.bam$' | wc -l)

for bam in *sorted.bam
do
  [[ "$bam" == "IgG.sorted.bam" ]] && continue

  count=$((count+1))
  # remove sorted bam files, name will be TAZ.sorted.bam -> TAZ
  name="${bam%.sorted.bam}"
  # folder will be TAZ_peak
  outdir="${name}_peak"

  # will only print out the commands
  echo "[$count/$total] Calling peaks for $name"

  macs3 callpeak \
    -t "$bam" \
    -c IgG.sorted.bam \
    -f BAM \
    -n "$name" \
    -g hs \
    --outdir "$outdir"
done


How many peaks we get for each transcription factor?

In [ ]:
find . -name "*Peak"  | xargs wc -l

# wc = what count
# for each file, how many lines there are

In [ ]:
# look at head file
# tells you the peaks, chromosome 1 is YAP_peak_1
# other statistics, p-values, q-values
head YAP_peak/YAP_peaks.narrowPeak

YAP_peaks.narrowPeak can be uploaded directly to IGV

According to the manual page:

NAME_peaks.narrowPeak is BED6+4 format file which contains the peak locations together with peak summit, p-value, and q-value. If you plan to load it to the UCSC genome browser, please make sure that you turn on –trackline option. Definition of some specific columns are:

5th: integer score for display. It’s calculated as int(-10log10pvalue) or int(-10log10qvalue) depending on whether -p (pvalue) or -q (qvalue) is used as score cutoff. Please note that currently this value might be out of the [0-1000] range defined in UCSC ENCODE narrowPeak format. You can let the value saturated at 1000 (i.e. p/q-value = 10^-100) by using the following 1-liner awk: awk -v OFS=“ ‘{$5=$5>1000?1000:$5} {print}’ NAME_peaks.narrowPeak

7th: fold-change at peak summit

8th: -log10pvalue at peak summit

9th: -log10qvalue at peak summit

10th: relative summit position to peak start

##### What is Model Building in MACS?

Model building in MACS (Model-Based Analysis of ChIP-Seq) is a step that attempts to determine the average fragment size of DNA fragments from sequencing data. It uses the shifted positions of the tags (forward and reverse reads) to estimate the peak signal. The estimated fragment size is then used to build a model of the peak shape, which is important for identifying and refining peak regions.

##### Model Building: Single-End vs. Paired-End

- Single-End Data:<br>
Model building is used because the fragment size is not directly available from single-end reads. MACS shifts the reads by half the estimated fragment size to align them to the putative binding sites.

- Paired-End Data:<br>
Model building is usually not needed because the fragment size is directly available from the paired-end read alignment. In paired-end mode, MACS calculates the actual insert size between read pairs, bypassing the need for model building.

##### When to Use –no-model and –extend-size
For single-end data, you might use –no-model and –extend-size in specific scenarios where you want to skip model building and manually specify the fragment size:

–no-model:

Disables the model building step. Use this option if you already know the approximate fragment size from your library preparation.

–extend-size:

Extends reads to the specified fragment size (e.g., 200 bp). This mimics the actual fragment length in the absence of paired-end information.

##### Best Practices for Single-End Data

With Model Building (Default Behavior):
Let MACS build the model unless there are specific reasons to disable it.

macs2 callpeak -t treatment.bam -c control.bam -f BED -g hs -n sample –outdir results

Without Model Building (–no-model):
Use when: You know the fragment size (e.g., from library preparation or empirical testing). The library is not suitable for accurate model building (e.g., very low read depth or noisy data). Specify –extend-size to set the fragment size.

macs2 callpeak -t treatment.bam -c control.bam -f BED -g hs -n sample --outdir results --no-model --extend-size 200

##### Why Use –no-model and –extend-size for Single-End?

Consistency: You can ensure the fragment size is consistently applied across all datasets. Noise Reduction: Avoid errors from noisy or low-quality data affecting model estimation. Speed: Skipping model building can make peak calling faster.

Model building is relevant for single-end data and used by default unless –no-model is specified. For paired-end data, model building is typically unnecessary since the fragment size is directly calculated. Use –no-model –extend-size for single-end data when you want to bypass model building and directly set the fragment size.

##### Challenges

The paper uses a previous generated ChIP-seq dataset https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE49651 for H3K4me1, H3K4me3 and H3K27ac. The authors used H3K4me1 to define enhancers and H3K4me3 to define promoters. H3K4me1/H3K27ac to define active enhancers and H3K4me3/H3K27ac to define active promoters.

Can you process the data yourself?

## reproduce figure1 a b c

https://crazyhottommy.github.io/reproduce_genomics_paper_figures/04_figure1_a_b_c.html

a) Venn-diagram
    - Need 3 numbers:
        1) Set A
        2) Set B
        3) Intersection

b) Venn-diagram

c) Line Plot
    - What's on the X-Axis and what's on the Y-Axis?
        - X-Axis: Distance to the summit of TAZ peaks (bp)
                  So TADS1 is one of the transferring factors that they profile using ChIP-seq.
        - Y-Axis: Peak density, so it's in the position of the TAD4 peak summit.
    - What data are needed to make those figures?

     So essentially, you, looking at the TADS peak, summit, and you look at the TAD4, the other transferring factor, the summit along the summit of TADS, and then, across all the peaks, then you can essentially build the line graph.

Figure 1a and 1b are venn-diagram showing the overlapping peak number of YAP/TAZ and YAP/TAZ/TEAD4

To find overlap/intersection of genomic regions, one can use the famous [bedtools](https://bedtools.readthedocs.io/en/latest/index.html) on command line or use `GenomicRanges` in R.

###### How do you find/get the numbers for the Venn-Diagram

It's a matter of finding intersections

#### Let's try bedtools

In [ ]:
conda install bedtools

`bedtools intersect` is one of the most frequently used

Fot CHiP-seq data you identify a Peakset of A and Peakset B.
You can use the intersect A and B to find intersectiones.

In [ ]:
# look at the lines
head TAZ_peaks.narrowPeak

# first 10 lines
# regular text file
# there is a bat file format with three colons, most important one
# Chromosome struck, name and the columns


In [ ]:
cd data/fatstq

bedtools intersect -a TAZ_peak/TAZ_peaks.narrowPeak -b YAP_peak/YAP_peaks.narrowPeak -wa | wc -l

7199

There are 7199 TAZ peaks overlap with YAP1 peaks.

I usually pipe the output to sort | uniq to get the unique TAZ peaks because there are could be one TAZ peak overlapping with mulitiple YAP1 peaks and it gets repeated in the bedtools output.

In [ ]:
bedtools intersect -a TAZ_peak/TAZ_peaks.narrowPeak -b YAP_peak/YAP_peaks.narrowPeak -wa | sort | uniq |  wc -l

There are 7154 unique TAZ peaks overlap with YAP1 peaks.

###### Use GenomicRanges in R

Let’s use R instead with the `GenomicRanges` package. Read my blog post on why we need to learn how to use it.

In [ ]:
# will return you all the peaks in A (TAZ_peak) that is overlapping with B (YAP_peak)
bedtools intersect -a TAZ_peak/TAZ_peaks.narrowPeak -b YAP_peak/YAP_peaks.narrowPeak -wa | head

In [ ]:
# if you do this sort first & uniq
# it will only give you the uniq rows
# wc = what count
# -l = how many lines there are

bedtools intersect -a TAZ_peak/TAZ_peaks.narrowPeak -b YAP_peak/YAP_peaks.narrowPeak -wa | sort | uniq |  wc -l

There are 7154 unique TAZ peaks overlap with YAP1 peaks.

They are ~45 lines less, because some of the peaks, a single peak, can overlap with multiple peaks in B peaks Set

### Use GenomicRanges in R

Let’s use R instead with the `GenomicRanges` package. Read my [blog post](https://divingintogeneticsandgenomics.com/post/genomic-interval/) on why we need to learn how to use it.

In [ ]:
-> See Fig_1_a_b_c.r

Tip: how do I what packages to use? google “plotting genome tracks bioconductor” or ask ChatGPT!!

# Stacked Barplot

y-axis = percentage of peaks